In [3]:
from anytree import PostOrderIter
from anytree.importer import DictImporter


In [2]:
!pip install anytree

  Obtaining dependency information for anytree from https://files.pythonhosted.org/packages/6a/fb/ff946843e6b55ae9fda84df3964d6c233cd2261dface789f5be02ab79bc5/anytree-2.12.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.9 kB ? eta -:--:--
   ---------------------------------------- 44.9/44.9 kB 1.1 MB/s eta 0:00:00


In [4]:
#read file
import json
with open('flare.json') as f:
    js_data = json.loads(f.read())

In [5]:
#imports dictonary in a tree form
import pandas as pd
import plotly
import plotly.graph_objects as go
from ipywidgets import widgets
import plotly.io as pio
pio.renderers.default = 'iframe'
importer = DictImporter()
root = importer.import_(js_data)


size = []
name = []
parent = []
level = []

def format(node):
  for i in node.children:
    #check if node as attribute value
    if hasattr(i, 'value') == False:
      format(i)
    v = i.value
    #check if node parent as attribute value
    if hasattr(i.parent, 'value'):
      i.parent.value += v
    #if node parent doesn't have a value set to same val as child
    elif hasattr(i.parent, 'value')== False:
      i.parent.value = v

    level.append(len(i.ancestors))
    name.append(i.name)
    parent.append(i.parent.name)
    size.append(i.value)
    
format(root)

#append attributes for root
level.append(0)
name.append(root.name)
parent.append("")
size.append(root.value)

#create df
df = pd.DataFrame()
df['parent'] = parent
df['name'] = name
df['value']= size
df['level'] = level

#slider funtion
def update(sliderVal):
  fig = plotly.graph_objs.Figure()
  fig.add_trace(plotly.graph_objs.Treemap(
      labels = df[df['level']<sliderVal]['name'],
      values = df[df['level']<sliderVal]['value'],
      parents = df[df['level']<sliderVal]['parent']  
  ))
  fig.update_traces(root_color="#f1f1f1")
  fig.update_layout(width = 900, height = 900)
  fig.show()

#create slider widget
widgets.interact(update, sliderVal = (0, 5))

interactive(children=(IntSlider(value=2, description='sliderVal', max=5), Output()), _dom_classes=('widget-int…

<function __main__.update(sliderVal)>